In [11]:
%%capture
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use("bmh")
plt.rcParams["axes.titlesize"] = "large"
import os
import tqdm
import pathlib
import numpy as np
import pandas as pd

tqdm.tqdm.pandas()
pd.set_option('display.max_columns', None)
import sklearn
from sklearn.metrics import mean_absolute_percentage_error

#дополнительно
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor
import joblib
!pip install fastparquet

In [18]:
named_models['target0_gas1']

VotingRegressor(estimators=[('XGBRegressor',
                             XGBRegressor(base_score=None, booster='gbtree',
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=0.1, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=10,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=300, n_jobs=None,
                                          num_parallel_tree=None,
                                          predictor=None, random_state=None, ...)),
                            ('ExtraTreesRegressor',
                             ExtraTreesRegressor(max_depth=15,
                                                 n_estimators=300))])

In [ ]:
group_lists = {}
statistics = train.groupby('feature4').describe()
for index, row in statistics.iterrows():
    group_name = index
    var_list = []
    
    for col in statistics.loc[index].index:
        if col[1] == 'std' and statistics.loc[index][col] < 1:
            var_list.append(col[0])
    
    group_lists[group_name] = var_list
    


# Создаём сабмит

In [23]:
from sklearn.decomposition import PCA

def apply_pca(df):
    gas1_df = df[df['feature4'] == 'gas1'].copy()
    gas2_df = df[df['feature4'] == 'gas2'].copy()

    # PCA for gas1
    if not gas1_df.empty:
        gas1_features = ['feature0', 'feature2', 'feature12', 'feature15', 'feature17', 'feature18', 'feature22', 'feature23']
        pca_gas1 = PCA(n_components=3)
        gas1_pca = pca_gas1.fit_transform(gas1_df[gas1_features])
        gas1_df[['pca1', 'pca2', 'pca3']] = pd.DataFrame(gas1_pca, index=gas1_df.index)

    # PCA for gas2
    if not gas2_df.empty:
        gas2_features = ['feature0', 'feature8', 'feature10', 'feature12', 'feature19', 'feature23', 'feature24']
        pca_gas2 = PCA(n_components=3)
        gas2_pca = pca_gas2.fit_transform(gas2_df[gas2_features])
        gas2_df[['pca1', 'pca2', 'pca3']] = pd.DataFrame(gas2_pca, index=gas2_df.index)

    combined_df = pd.concat([gas1_df, gas2_df])
    combined_df.sort_index(inplace=True)

    return combined_df


In [26]:
df_test

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,target0,target1
0,32.910908,17.376350,77.557840,2.929855,gas1,20.487150,57.633085,49.245392,-44.124381,23.400064,-28.640648,-10.995420,40.856313,29.912800,33.168114,18.769718,41.812507,14.203652,9.689942,17.951627,24.198589,102.448710,110.529868,56.817260,12.887802,27.050891,6.502743
1,41.263782,22.419445,47.945514,-25.847472,gas2,21.461239,3.474080,49.659980,-13.553188,-13.047593,-12.445940,16.657368,42.453280,99.792375,127.159321,-13.180244,50.722470,-14.451904,-0.906120,105.724742,-9.435222,-16.060150,37.286110,61.224272,39.822424,84.127890,76.578716
2,25.580283,17.376350,77.654180,2.799411,gas1,20.487150,28.938295,49.245392,-44.124381,20.610679,-28.641150,-1.886830,40.927941,16.340674,60.760647,18.658501,52.731646,14.395054,9.779019,17.951638,24.999453,101.728213,111.471534,52.664304,12.887802,22.080133,3.036043
3,33.756900,17.376350,73.049625,2.953982,gas1,20.487150,28.932311,49.245392,-44.124381,18.107963,-28.642094,16.284415,40.869406,43.707663,48.037510,18.771030,52.305692,14.184314,9.798969,17.951675,23.990300,101.312113,115.589451,56.840719,12.887802,30.234082,8.910795
4,4.223732,38.772534,48.015553,-25.843943,gas2,24.635721,12.011581,51.030938,84.244199,-17.735680,40.395582,16.557978,65.988300,91.081310,150.949700,-13.172169,51.012294,-14.439953,8.262354,90.187207,-2.901661,-16.060150,37.398779,40.488468,128.295838,71.128092,50.475082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153412,25.851721,17.376350,69.777513,3.086227,gas1,20.487150,9.187947,49.245392,-44.124381,8.395057,-28.641157,25.687912,40.874155,30.217753,60.629469,18.846511,74.241020,14.248014,9.806796,17.951871,24.302496,101.668459,119.189777,52.544704,12.887802,26.319008,4.318349
153413,33.211644,17.376350,84.113483,2.963000,gas1,20.487150,37.677592,49.245392,-44.124381,6.252886,-28.641255,99.694891,40.855835,29.957800,59.524126,18.828185,42.093082,14.351076,9.848909,17.951828,24.122972,101.455016,103.624989,56.818116,12.887802,19.951185,7.128586
153414,0.339018,17.376350,77.627839,2.889655,gas1,20.487150,49.094915,49.245392,-44.124381,23.596678,-28.640526,25.816561,40.866704,43.946905,60.819833,19.073488,85.380269,14.402462,9.798969,17.951898,16.179377,101.198059,110.992570,39.673505,12.887802,28.492906,5.589999
153415,21.018093,38.902218,48.015239,-25.843954,gas2,24.642608,0.626815,51.022655,84.812493,-30.045713,40.703237,-1.855328,65.997626,88.992559,126.985525,-13.172113,72.374653,-14.439792,8.337519,89.157475,-2.915824,-16.060150,37.399312,50.917348,128.621094,72.065991,51.306816


In [25]:
apply_pca(df_test)

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,target0,target1,pca1,pca2,pca3
0,32.910908,17.376350,77.557840,2.929855,gas1,20.487150,57.633085,49.245392,-44.124381,23.400064,-28.640648,-10.995420,40.856313,29.912800,33.168114,18.769718,41.812507,14.203652,9.689942,17.951627,24.198589,102.448710,110.529868,56.817260,12.887802,27.050891,6.502743,19.674097,-2.299538,1.296653
1,41.263782,22.419445,47.945514,-25.847472,gas2,21.461239,3.474080,49.659980,-13.553188,-13.047593,-12.445940,16.657368,42.453280,99.792375,127.159321,-13.180244,50.722470,-14.451904,-0.906120,105.724742,-9.435222,-16.060150,37.286110,61.224272,39.822424,84.127890,76.578716,77.269214,-20.846219,27.285959
2,25.580283,17.376350,77.654180,2.799411,gas1,20.487150,28.938295,49.245392,-44.124381,20.610679,-28.641150,-1.886830,40.927941,16.340674,60.760647,18.658501,52.731646,14.395054,9.779019,17.951638,24.999453,101.728213,111.471534,52.664304,12.887802,22.080133,3.036043,11.337947,-1.110233,0.895696
3,33.756900,17.376350,73.049625,2.953982,gas1,20.487150,28.932311,49.245392,-44.124381,18.107963,-28.642094,16.284415,40.869406,43.707663,48.037510,18.771030,52.305692,14.184314,9.798969,17.951675,23.990300,101.312113,115.589451,56.840719,12.887802,30.234082,8.910795,18.982585,-9.075466,1.054279
4,4.223732,38.772534,48.015553,-25.843943,gas2,24.635721,12.011581,51.030938,84.244199,-17.735680,40.395582,16.557978,65.988300,91.081310,150.949700,-13.172169,51.012294,-14.439953,8.262354,90.187207,-2.901661,-16.060150,37.398779,40.488468,128.295838,71.128092,50.475082,-65.481950,3.101652,-15.470756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153412,25.851721,17.376350,69.777513,3.086227,gas1,20.487150,9.187947,49.245392,-44.124381,8.395057,-28.641157,25.687912,40.874155,30.217753,60.629469,18.846511,74.241020,14.248014,9.806796,17.951871,24.302496,101.668459,119.189777,52.544704,12.887802,26.319008,4.318349,9.183471,-11.913267,0.564432
153413,33.211644,17.376350,84.113483,2.963000,gas1,20.487150,37.677592,49.245392,-44.124381,6.252886,-28.641255,99.694891,40.855835,29.957800,59.524126,18.828185,42.093082,14.351076,9.848909,17.951828,24.122972,101.455016,103.624989,56.818116,12.887802,19.951185,7.128586,21.938253,6.940890,1.769714
153414,0.339018,17.376350,77.627839,2.889655,gas1,20.487150,49.094915,49.245392,-44.124381,23.596678,-28.640526,25.816561,40.866704,43.946905,60.819833,19.073488,85.380269,14.402462,9.798969,17.951898,16.179377,101.198059,110.992570,39.673505,12.887802,28.492906,5.589999,-16.269987,5.245449,-0.069477
153415,21.018093,38.902218,48.015239,-25.843954,gas2,24.642608,0.626815,51.022655,84.812493,-30.045713,40.703237,-1.855328,65.997626,88.992559,126.985525,-13.172113,72.374653,-14.439792,8.337519,89.157475,-2.915824,-16.060150,37.399312,50.917348,128.621094,72.065991,51.306816,-66.257941,3.634379,4.267385


In [35]:
# %%writefile /kaggle/input/sibur23/predict.py

def apply_pca(df):
    gas1_df = df[df['feature4'] == 'gas1'].copy()
    gas2_df = df[df['feature4'] == 'gas2'].copy()

    # PCA for gas1
    if not gas1_df.empty:
        gas1_features = ['feature0', 'feature2', 'feature12', 'feature15', 'feature17', 'feature18', 'feature22', 'feature23']
        pca_gas1 = PCA(n_components=3)
        gas1_pca = pca_gas1.fit_transform(gas1_df[gas1_features])
        gas1_df[['pca1', 'pca2', 'pca3']] = pd.DataFrame(gas1_pca, index=gas1_df.index)

    # PCA for gas2
    if not gas2_df.empty:
        gas2_features = ['feature0', 'feature8', 'feature10', 'feature12', 'feature19', 'feature23', 'feature24']
        pca_gas2 = PCA(n_components=3)
        gas2_pca = pca_gas2.fit_transform(gas2_df[gas2_features])
        gas2_df[['pca1', 'pca2', 'pca3']] = pd.DataFrame(gas2_pca, index=gas2_df.index)

    combined_df = pd.concat([gas1_df, gas2_df])
    combined_df.sort_index(inplace=True)

    return combined_df

def load_and_rename_models(file_path, model_names):
    loaded_models = joblib.load(file_path + 'ensemble_models.joblib')
    named_models = dict(zip(model_names, loaded_models))
    return named_models

file_path = '/kaggle/input/sibur23/'
model_names = ['target0_gas1', 'target1_gas1', 'target0_gas2', 'target1_gas2']
named_models = load_and_rename_models(file_path, model_names)

def get_model(feature4, target):
    """
    Выбор модели на основе комбинации feature4 и target.

    Параметры:
        feature4 (str): Значение feature4 ('gas1' или 'gas2').
        target (str): Имя целевого столбца ('target0' или 'target1').

    Возвращает:
        Модель, соответствующую комбинации feature4 и target.
    """

    if feature4 == 'gas1' and target == 'target0':
        return named_models['target0_gas1']
    elif feature4 == 'gas1' and target == 'target1':
        return named_models['target1_gas1']
    elif feature4 == 'gas2' and target == 'target0':
        return named_models['target0_gas2']
    elif feature4 == 'gas2' and target == 'target1':
        return named_models['target1_gas2']
    else:
        raise ValueError(f"Некорректная комбинация feature4 и target: {feature4}, {target}")

def predict(df: pd.DataFrame) -> pd.DataFrame:
    """
    Вычисление предсказаний.
    Параметры:
        df:
          датафрейм, содержащий строки из тестового множества.
    Результат:
        Датафрейм предсказаний.
    """
    df = apply_pca(df)
    gas1_features_to_remove = ['feature1', 'feature5', 'feature7', 'feature8', 'feature10', 'feature19', 'feature24', 'feature0', 'feature2', 'feature12', 'feature15', 'feature17', 'feature18', 'feature22', 'feature23']
    gas2_features_to_remove = ['feature2', 'feature3', 'feature15', 'feature17', 'feature21', 'feature22', 'feature0', 'feature8', 'feature10', 'feature12', 'feature19', 'feature23', 'feature24']
    predictions = pd.DataFrame(index=df.index, columns=["target0", "target1"])
    df = df.drop(columns=["target0", "target1"])

    for index, row in df.iterrows():
        feature4 = row["feature4"]
        row = row.drop("feature4")
        target0 = "target0"
        target1 = "target1"
        
        model_target0 = get_model(feature4, target0)
        model_target1 = get_model(feature4, target1)
        
        # Remove features
        if feature4 == "gas1":
            row.drop(gas1_features_to_remove, inplace=True)
        elif feature4 == "gas2":
            row.drop(gas2_features_to_remove, inplace=True)
        
        if model_target0 is not None and feature4 == "gas1":
            pred_target0 = model_target0.predict(row.values.reshape(1, -1))
            predictions.loc[index, "target0"] = pred_target0[0]
        
        if model_target1 is not None and feature4 == "gas1":
            pred_target1 = model_target1.predict(row.values.reshape(1, -1))
            predictions.loc[index, "target1"] = pred_target1[0]
            
        if model_target0 is not None and feature4 == "gas2":
            pred_target0 = model_target0.predict(row.values.reshape(1, -1))
            predictions.loc[index, "target0"] = pred_target0[0]
            
        if model_target1 is not None and feature4 == "gas2":
            pred_target1 = model_target1.predict(row.values.reshape(1, -1))
            predictions.loc[index, "target1"] = pred_target1[0]

    return predictions

In [36]:
df_test = pd.read_parquet("/kaggle/input/sibur23/train.parquet")

predictions = predict(df_test)

# Расчет MAPE для target0
actual_target0 = df_test["target0"]
predicted_target0 = predictions["target0"]
mape_target0 = np.mean(np.abs((actual_target0 - predicted_target0) / actual_target0)) * 100

# Расчет MAPE для target1
actual_target1 = df_test["target1"]
predicted_target1 = predictions["target1"]
mape_target1 = np.mean(np.abs((actual_target1 - predicted_target1) / actual_target1)) * 100

# Расчет общего MAPE
mape_total = (mape_target0 + mape_target1) / 2

# Вывод общего MAPE с округлением до 2 знаков после запятой
print("Total MAPE: {:.2f}%".format(mape_total))

ValueError: Feature shape mismatch, expected: 13, got 12